In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

In [4]:
# Langchain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader


# Vector store
from langchain_community.vectorstores import Chroma


# Utility Imports
import numpy as np
from typing import List, Any, Dict

### 1.  Create sample data


In [5]:
# Create sample documents

## create sample documents
sample_docs = [
    """
    Machine Learning Fundamentals
    
    Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine learning: supervised learning, unsupervised learning, and reinforcement 
    learning. Supervised learning uses labeled data to train models, while unsupervised 
    learning finds patterns in unlabeled data. Reinforcement learning learns through 
    interaction with an environment using rewards and penalties.
    """,
    
    """
    Deep Learning and Neural Networks
    
    Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning has revolutionized fields like computer vision, natural language 
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly 
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers 
    excel at sequential data processing.
    """,
    
    """
    Natural Language Processing (NLP)
    
    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, 
    machine translation, and question answering. Modern NLP heavily relies on transformer 
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand 
    context and relationships between words in text.
    """
]

sample_docs


['\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    ',
 '\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective f

In [ ]:
## Save the sample docss to files
import tempfile

tempdir =  tempfile.mkdtemp()

for i, doc in enumerate(sample_docs):
    with open(f"sample_doc_{i}.txt", "w") as f:
        f.write(doc)



In [15]:
tempdir

'C:\\Users\\iabhi\\AppData\\Local\\Temp\\tmpvxb726e6'

### 2. Docuemtnt Loading

In [14]:
from langchain.document_loaders import DirectoryLoader

In [21]:
# Load all the matching documents from the directory

loader = DirectoryLoader(
    path= "/",
    glob= "*.txt",
    loader_cls= TextLoader,
    show_progress= True,
    loader_kwargs= {"encoding": "utf-8"}
) 

In [25]:
docs = loader.load()
docs

100%|██████████| 3/3 [00:00<00:00, 3984.46it/s]


[Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through \n    interaction with an environment using rewards and penalties.\n    '),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision,

### 3. Document Splitting

In [ ]:
# Initialize the text splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap= 50,
    length_function= len, 
    separators= ["\n\n", "\n", " ", ""]
)



In [29]:
chunks   =  text_splitter.split_documents(docs)
chunks

[Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural netwo

In [30]:
len(chunks)

7

### 4. Embedding Model

In [32]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000018799BE9F90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000018799BEA850>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

### 5. Vector Store - Chroma DB

In [33]:
## Create a chroma db vector store

persist_directory = "./chroma_db"


# Initialize the chroma db with openai embeddings

chroma_db = Chroma.from_documents(
    documents= chunks,
    embedding= embeddings_model,
    persist_directory= persist_directory, 
    collection_name= "rag_collection"
)

chroma_db

### 6. Test Similarity Search

In [34]:
query = "What are the types of machine learning?"

search_results = chroma_db.similarity_search(query, k=3)
search_results

[Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Con

In [35]:
query = "What is nlp?"

search_results = chroma_db.similarity_search(query, k=3)
search_results

[Document(metadata={'source': '\\sample_doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand \n    context and relationships between words in text.'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, 

In [36]:
print(f"Query: {query}")
print(f"\nTop {len(search_results)} similar chunks:")
for i, doc in enumerate(search_results):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:200] + "...")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")

Query: What is nlp?

Top 3 similar chunks:

--- Chunk 1 ---
Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language. 
    Key tasks in NLP include text classification, named entity recogn...
Source: \sample_doc_2.txt

--- Chunk 2 ---
Deep Learning and Neural Networks...
Source: \sample_doc_1.txt

--- Chunk 3 ---
Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning...
Source: \sample_doc_1.txt


### Advanced similarity search with scores

In [37]:
search_results =  chroma_db.similarity_search_with_score(query, k=3)
search_results

[(Document(metadata={'source': '\\sample_doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand \n    context and relationships between words in text.'),
  0.7475031614303589),
 (Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
  1.2537007331848145),
 (Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like com

### 7. Initialize LLM Model

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-nano")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001879A505E50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001879A5060D0>, root_client=<openai.OpenAI object at 0x000001879A4B2650>, root_async_client=<openai.AsyncOpenAI object at 0x000001879A4B2780>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [40]:
llm.invoke("What is LLM")

AIMessage(content='LLM stands for Large Language Model. It’s an AI model trained on extremely large amounts of text to understand and generate human-like language. Key points:\n\n- Built with transformer architectures and trained to predict the next word (or fill in tokens) given a context.\n- Typically pre-trained on broad text data, then fine-tuned or instruction-tuned for specific tasks.\n- Able to perform many language tasks: writing, summarization, translation, answering questions, coding, chat, and more.\n- They come in different sizes (billions to trillions of parameters) and can be decoder-only (GPT-style) or encoder-decoder (like some T5/UX models). BERT is an encoder-only example; GPT-4 is a decoder-style example.\n- Strengths: impressive generation and understanding across many domains.\n- Limitations: can hallucinate facts, reflect biases in training data, have knowledge cutoffs, and may require safeguards.\n\nIf you want, I can tailor the explanation to a specific model ex

In [42]:
from langchain.chat_models.base import init_chat_model

llm_2 = init_chat_model("openai:gpt-5-nano")

In [44]:
llm_2.invoke("What is LLM")

AIMessage(content='LLM can mean two different things. Which one did you have in mind?\n\n1) Large Language Model (AI)\n- What it is: An AI system built to understand and generate human-like text. It’s a type of machine learning model, usually based on transformers, trained on huge amounts of text from the web, books, and other sources.\n- How it works (high level): It learns patterns in language from vast data, then predicts the next word or sequence of words to generate coherent text. It can also perform tasks like summarization, translation, question answering, and code generation.\n- Capabilities: Text generation, editing, summarization, translation, reasoning within some limits, coding help, conversational chat.\n- Limitations: Can produce plausible-sounding but incorrect or biased content (hallucinations), reflects biases in training data, requires substantial compute and data privacy considerations, not truly understanding or conscious.\n- Common uses: Chatbots, virtual assistant

### 8. Modern RAG Chain

In [47]:
from langchain.chains import  create_retrieval_chain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [48]:
## Convert vector store to retriever

retriever = chroma_db.as_retriever(
    search_kwargs= {"k": 3} # Retrieve top 3 results
 )

retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3})

In [49]:
# Create a prompt template

system_prompt="""You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [50]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [51]:
### Create a document chain

document_chain = create_stuff_documents_chain(
    llm= llm,
    prompt= prompt
)

document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001879A505E50>, async_client=<openai.res

In [53]:
## Create the final rag chain

rag_chain = create_retrieval_chain(
    retriever,
    document_chain
)

rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf yo

In [54]:
rag_chain.invoke({"input": "What is NLP?"})  

{'input': 'What is NLP?',
 'context': [Document(metadata={'source': '\\sample_doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language. \n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis, \n    machine translation, and question answering. Modern NLP heavily relies on transformer \n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand \n    context and relationships between words in text.'),
  Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
  Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vis

In [55]:
# Function to query the modern RAG system
def query_rag_modern(question):
    print(f"Question: {question}")
    print("-" * 50)
    
    # Using create_retrieval_chain approach
    result = rag_chain.invoke({"input": question})
    
    print(f"Answer: {result['answer']}")
    print("\nRetrieved Context:")
    for i, doc in enumerate(result['context']):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200] + "...")
    
    return result

# Test queries
test_questions = [
    "What are the three types of machine learning?",
    "What is deep learning and how does it relate to neural networks?",
    "What are CNNs best used for?"
]

for question in test_questions:
    result = query_rag_modern(question)
    print("\n" + "="*80 + "\n")

Question: What are the three types of machine learning?
--------------------------------------------------
Answer: The three main types are supervised learning, unsupervised learning, and reinforcement learning.

Retrieved Context:

--- Source 1 ---
Machine learning is a subset of artificial intelligence that enables systems to learn 
    and improve from experience without being explicitly programmed. There are three main 
    types of machine l...

--- Source 2 ---
Machine Learning Fundamentals...

--- Source 3 ---
Deep learning is a subset of machine learning based on artificial neural networks. 
    These networks are inspired by the human brain and consist of layers of interconnected 
    nodes. Deep learning...


Question: What is deep learning and how does it relate to neural networks?
--------------------------------------------------
Answer: Deep learning is a subset of machine learning that uses artificial neural networks with many layers to learn hierarchical representations

## Create RAG using LCEL

In [57]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel



In [59]:
# Create a custom prompt
custom_prompt = ChatPromptTemplate.from_template("""Use the following context to answer the question. 
If you don't know the answer based on the context, say you don't know.
Provide specific details from the context to support your answer.

Context:
{context}

Question: {question}

Answer:"""
)


custom_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. \nIf you don't know the answer based on the context, say you don't know.\nProvide specific details from the context to support your answer.\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])

In [60]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [73]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3})

In [74]:
# Build the chain using LCEL

rag_chain_lcel = ( 
    {"context":retriever | format_docs, 
    "question": RunnablePassthrough()
    } 
    |
    custom_prompt
    |
    llm
    |
    StrOutputParser()
)

In [75]:
rag_chain_lcel

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following context to answer the question. \nIf you don't know the answer based on the context, say you don't know.\nProvide specific details from the context to support your answer.\n\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001879A505E50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001879A5060

In [77]:
response=rag_chain_lcel.invoke("What is Deep Learning")
response

'Deep Learning is a subset of machine learning based on artificial neural networks. These networks are inspired by the human brain and consist of layers of interconnected nodes. It has revolutionized fields like computer vision, natural language processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers are other common architectures.'

In [78]:
retriever.get_relevant_documents("What is Deep Learning")

C:\Users\iabhi\AppData\Local\Temp\ipykernel_21476\458153282.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is Deep Learning")


[Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural networks. \n    These networks are inspired by the human brain and consist of layers of interconnected \n    nodes. Deep learning has revolutionized fields like computer vision, natural language \n    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly \n    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine Learning Fundamentals')]

In [79]:
# Query using the LCEL approach - Fixed version
def query_rag_lcel(question):
    print(f"Question: {question}")
    print("-" * 50)
    
    # Method 1: Pass string directly (when using RunnablePassthrough)
    answer = rag_chain_lcel.invoke(question)
    print(f"Answer: {answer}")
    
    # Get source documents separately if needed
    docs = retriever.get_relevant_documents(question)
    print("\nSource Documents:")
    for i, doc in enumerate(docs):
        print(f"\n--- Source {i+1} ---")
        print(doc.page_content[:200] + "...")

In [80]:
# Test LCEL chain
print("Testing LCEL Chain:")
query_rag_lcel("What are the key concepts in reinforcement learning?")

Testing LCEL Chain:
Question: What are the key concepts in reinforcement learning?
--------------------------------------------------
Answer: - The key concept is interaction with an environment using rewards and penalties. In reinforcement learning, an agent acts in an environment and receives rewards or penalties that guide learning.

Source Documents:

--- Source 1 ---
Machine Learning Fundamentals...

--- Source 2 ---
interaction with an environment using rewards and penalties....

--- Source 3 ---
Deep Learning and Neural Networks...


## Add new documents to the vector store

In [81]:
# Add new documents to the existing vector store
new_document = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make 
decisions by interacting with an environment. The agent receives rewards or penalties 
based on its actions and learns to maximize cumulative reward over time. Key concepts 
in RL include: states, actions, rewards, policies, and value functions. Popular RL 
algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and 
Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), 
robotics, and autonomous systems.
"""

In [82]:
chunks

[Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, while unsupervised \n    learning finds patterns in unlabeled data. Reinforcement learning learns through'),
 Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': '\\sample_doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural netwo

In [83]:
new_doc=Document(
    page_content=new_document,
    metadata={"source": "manual_addition", "topic": "reinforcement_learning"}
)

In [84]:
new_doc

Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and \nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), \nrobotics, and autonomous systems.\n')

In [85]:
## Add new documents to the existing vector store
chroma_db

In [86]:
new_docs_chunks = text_splitter.split_documents([new_doc])
new_docs_chunks

[Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Reinforcement Learning in Detail'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Reinforcement learning (RL) is a type of machine learning where an agent learns to make \ndecisions by interacting with an environment. The agent receives rewards or penalties \nbased on its actions and learns to maximize cumulative reward over time. Key concepts \nin RL include: states, actions, rewards, policies, and value functions. Popular RL \nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and'),
 Document(metadata={'source': 'manual_addition', 'topic': 'reinforcement_learning'}, page_content='Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), \nrobotics, and autonomous systems.')]

In [87]:
chroma_db.add_documents(new_docs_chunks)

['e3e439bf-e0db-4573-9c8b-f1541336e65c',
 '43f38094-4b1d-4b88-b1e9-b8669631433f',
 '176de424-7fb4-4a37-a89d-d052a2bd70de']

In [88]:
print(f"Added {len(new_docs_chunks)} new chunks to the vector store")
print(f"Total vectors now: {chroma_db._collection.count()}")    

Added 3 new chunks to the vector store
Total vectors now: 10


In [89]:
## query with the updated vector
new_question="What are the keys concepts in reinforcement learning"
result=query_rag_lcel(new_question)
result

Question: What are the keys concepts in reinforcement learning
--------------------------------------------------
Answer: - States
- Actions
- Rewards
- Policies
- Value functions

These concepts describe the environment (states), the agent’s choices (actions), the feedback signal (rewards), how the agent behaves (policies), and the long-term value of states or actions (value functions), with the goal of maximizing cumulative rewards over time.

Source Documents:

--- Source 1 ---
Reinforcement Learning in Detail...

--- Source 2 ---
Reinforcement learning (RL) is a type of machine learning where an agent learns to make 
decisions by interacting with an environment. The agent receives rewards or penalties 
based on its actions and...

--- Source 3 ---
Machine Learning Fundamentals...


## Advanced RAG Technique - Conversational Memory

In [90]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [91]:
## create a prompt that includes the chat history
contextualize_q_system_prompt = """Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question 
which can be understood without the chat history. Do NOT answer the question, 
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [92]:
## create history aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(ta

In [93]:
# Create a new document chain with history
qa_system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create conversational RAG chain
conversational_rag_chain = create_retrieval_chain(
    history_aware_retriever, 
    question_answer_chain
)
print("Conversational RAG chain created!")

Conversational RAG chain created!


In [94]:
conversational_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000018799B8D6A0>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')]

In [95]:
chat_history=[]
# First question
result1 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "What is machine learning?"
})
print(f"Q: What is machine learning?")
print(f"A: {result1['answer']}")

Q: What is machine learning?
A: Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses data to train models and includes three main types: supervised learning (labeled data), unsupervised learning (patterns in unlabeled data), and reinforcement learning (learning from interaction).


In [96]:
chat_history.extend([
    HumanMessage(content="What is machine learning"),
    AIMessage(content=result1['answer'])
])

In [97]:
chat_history

[HumanMessage(content='What is machine learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses data to train models and includes three main types: supervised learning (labeled data), unsupervised learning (patterns in unlabeled data), and reinforcement learning (learning from interaction).', additional_kwargs={}, response_metadata={})]

In [98]:
## Follow up question
result2 = conversational_rag_chain.invoke({
    "chat_history": chat_history,
    "input": "What are its main types?"  # Refers to ML from previous question
})
result2

{'chat_history': [HumanMessage(content='What is machine learning', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. It uses data to train models and includes three main types: supervised learning (labeled data), unsupervised learning (patterns in unlabeled data), and reinforcement learning (learning from interaction).', additional_kwargs={}, response_metadata={})],
 'input': 'What are its main types?',
 'context': [Document(metadata={'source': '\\sample_doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables systems to learn \n    and improve from experience without being explicitly programmed. There are three main \n    types of machine learning: supervised learning, unsupervised learning, and reinforcement \n    learning. Supervised learning uses labeled data to train models, whi